# <div align="center" style="color: #ff5733;">TSA Closure Third Notification</div>

In [1]:
# Import Libraries   

import pandas as pd  
import numpy as np  

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

# Ask Balasubramani to refresh risk_credit_mis.tsa_account_closure table.
**Check if that is refreshed when Balasubramani confirms it**

In [2]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID 906f84cc-3ceb-4211-8b89-8335e2e54bed successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2024-10-15,709821


The date should show yesterday's date

# Read the data from the first sheet for Type1 and Type2 data provided by Marvin

In [3]:
d1 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\September2024\ThirdNotification_files\Type 1 & 2 - Sept20241016.xlsx")
d1.shape

(25984, 2)

In [4]:
print(f"The unique count of customer_id is:\t{len(d1['Customer_Id'].unique())}")
d1.head()


The unique count of customer_id is:	25983


,Customer_Id,ACTION_TYPE
0,2601808,TYPE 1
1,2602003,TYPE 1
2,2629536,TYPE 1
3,2633912,TYPE 1
4,2661623,TYPE 1


# Read the data from the second sheet for Type3 data provided by Marvin

In [5]:
d2 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\September2024\ThirdNotification_files\Type 3 - Sept20241016.xlsx")
d2.shape

(887, 2)

In [6]:
print(f"The unique count of customer_id is:\t{len(d2['Customer_Id'].unique())}")
d2.head()

The unique count of customer_id is:	887


,Customer_Id,ACTION_TYPE
0,2414995,TYPE 3
1,2419261,TYPE 3
2,1070043,TYPE 3
3,2161071,TYPE 3
4,1869127,TYPE 3


In [7]:
result = pd.concat([d1, d2], axis=0, ignore_index=True)
result.shape

(26871, 2)

In [8]:
result.ACTION_TYPE.value_counts()

ACTION_TYPE
TYPE 1    25914
TYPE 3      887
TYPE 2       70
Name: count, dtype: int64

In [16]:
result[result['Customer_Id'] == '2414995']

,Customer_Id,ACTION_TYPE
25984,2414995,TYPE 3


In [10]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26871 entries, 0 to 26870
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  26871 non-null  int64 
 1   ACTION_TYPE  26871 non-null  object
dtypes: int64(1), object(1)
memory usage: 420.0+ KB


In [14]:
result['Customer_Id'] = result['Customer_Id'].astype(str)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26871 entries, 0 to 26870
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  26871 non-null  object
 1   ACTION_TYPE  26871 non-null  object
dtypes: object(2)
memory usage: 420.0+ KB


In [17]:
result.to_csv("temp.csv", index = False)

In [18]:
a = tuple(result['Customer_Id'])
a[:5]

('2601808', '2602003', '2629536', '2633912', '2661623')

In [19]:
d = '2024-10-15'   ### Need to change this to D-1 everytime we run this code ###
# Get the current date
current_date = pd.Timestamp.now().date()
d_minus_1_days = current_date - pd.DateOffset(days=1)
# Calculate D-60 days
d_minus_60_days = '2024-07-17'
# Calculate D-180 days
d_minus_180_days = '2024-03-19'

print("D-1 day from current date", d_minus_1_days )
print("D-60 day from the current date - 1:", d_minus_60_days)
print("D-180 day from the current date -1:", d_minus_180_days)

d_minus_1_days_string = d_minus_1_days.strftime('%Y-%m-%d')
print("D-1 day from the current date:", d_minus_1_days_string)

d_minus_60_days_string = d_minus_60_days
print("D-60 day from the current date:", d_minus_60_days_string)

d_minus_180_days_string = d_minus_180_days
print("D-180 day from the current date:", d_minus_180_days_string)

D-1 day from current date 2024-10-15 00:00:00
D-60 day from the current date - 1: 2024-07-17
D-180 day from the current date -1: 2024-03-19
D-1 day from the current date: 2024-10-15
D-60 day from the current date: 2024-07-17
D-180 day from the current date: 2024-03-19


Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [20]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}' ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
)
select * from w_tsa;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type =  'tqdm')
dfd.shape

Job ID 4ad01542-385b-4d8f-a2d2-e8774c335eae successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(26871, 29)

In [21]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in {a}
),
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select distinct
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '{d_minus_60_days_string}')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (coalesce(Active_Loan,0) = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'


    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '{d_minus_60_days_string}')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (coalesce(Active_Loan, 0) = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
            and (coalesce(Active_Loan, 0) = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
            and (Active_Term_Deposit_flag = 0)
            and TSA_Opening_date < date '{d_minus_180_days_string}'                                     ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and (Last_TSA_Credit_Date < date '{d_minus_180_days_string}'                                 ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and Last_TSA_Debit_Date < date '{d_minus_180_days_string}'                                         ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '{d_minus_180_days_string}' )  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and tsa_status = 'Active'
            and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
            and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
)
"""


In [22]:
print(sq)


with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2024-10-15'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  --customer_id in (select cast(customer_id as string) from prj-prod-dataplatform.dap_ds_poweruser_playground.tsa_2nd_notification_base_10jan2024)
  customer_id in ('2601808', '2602003', '2629536', '2633912', '2661623', '2535424', '2653067', '2612391', '2621030', '2625028', '2660890', '1963934', '2625058', '2577532', '2643609', '2636231', '2575382', '2617928', '2575592', '2596778', '2570839', '2664283', '2654450', '2570876', '2595251', '2613168', '2615200', '2623016', '2422326', '2579387', '2591167', '2634976', '2655588', '2619273', '2631404', '2645503', '2665723', '2585117',

In [23]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 66959cab-b398-4453-871a-3b3488081791 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [24]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
22249,2617518,2024-07-14,60826175180002,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-08-11,NaT,NaT,NaN,2024-10-15,Batch_1,October,0,1,NaT,2024-09-27,TYPE 1
16588,2662374,2024-07-15,60826623740009,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-07-15,NaT,NaT,NaN,2024-10-15,Batch_1,October,0,0,NaT,NaT,TYPE 1
15556,2633129,2024-07-04,60826331290007,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-07-05,NaT,NaT,NaN,2024-10-15,Batch_1,October,0,1,NaT,2024-10-03,TYPE 1
25152,2644581,2024-07-08,60826445810002,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-07-08,NaT,NaT,NaN,2024-10-15,Batch_1,October,0,1,NaT,2024-10-07,TYPE 1
18002,2661132,2024-07-15,60826611320009,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-07-15,NaT,NaT,NaN,2024-10-15,Batch_1,October,0,1,NaT,2024-10-14,TYPE 1
14638,2594680,2024-06-21,60825946800003,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,1,0,2024-09-11,NaT,NaT,NaN,2024-10-15,Batch_1,October,0,0,NaT,NaT,TYPE 1
7449,2599972,2024-06-22,60825999720002,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-06-22,NaT,NaT,NaN,2024-10-15,Batch_1,October,0,1,NaT,2024-09-21,TYPE 1
13830,2622034,2024-07-04,60826220340002,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-07-04,NaT,NaT,NaN,2024-10-15,Batch_1,October,0,1,NaT,2024-09-29,TYPE 1
11703,2616789,2024-06-28,60826167890009,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-06-28,NaT,NaT,NaN,2024-10-15,Batch_1,October,0,1,NaT,2024-09-27,TYPE 1
19690,2575396,2024-06-13,60825753960006,0.0,Active,0.0,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-06-11,NaT,NaT,NaN,2024-10-15,Batch_1,October,0,0,NaT,NaT,TYPE 1


In [25]:
df.shape

(26871, 32)

In [26]:
df['ACTION_TYPE'].value_counts()

ACTION_TYPE
TYPE 1       25781
TYPE 3         859
NO ACTION      162
TYPE 2          69
Name: count, dtype: int64

In [35]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\September2024\ThirdNotification_files\20241016_TSA_Account_Closure_3rdtNotification.xlsx", sheet_name='TSA_Closure_3rdNotification', index = False)

In [36]:
df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\September2024\ThirdNotification_files\20241016_TSA_Account_Closure_3rdtNotification.xlsx", sheet_name='TSA_Closure_3rdNotification')
df.shape

(26871, 32)

In [29]:
df.columns

Index(['Customer_Id', 'TSA_Opening_date', 'TSA_Account_Number',
       'OfblockAmount', 'TSA_Status', 'TSA_Balance', 'Solo_Stash_Balance',
       'Group_Stash_Balance', 'Term_Deposit_Balance',
       'Active_Term_Deposit_flag', 'Last_TSA_Credit_Date',
       'Last_TSA_Debit_Date', 'ARI_Flag', 'Active_Loan',
       'Total_Loan_Outstanding', 'Approved_Loans', 'Pending_Loan_Applications',
       'Hard_Reject_Loans_App', 'Soft_Reject_Loans_App', 'Cancelled_Loans_App',
       'Last_Login_Date', 'Last_TSA_EOD_Balance_Date',
       'Last_Group_Stash_Credit_Date', 'Group_Stash_Balance_Owner', 'Run_Date',
       'Batch', 'Processing_Month', 'pending_loans_not_expired_not_exempt',
       'expired_loans', 'pending_loans_lastDt', 'expired_loans_lastDt',
       'ACTION_TYPE'],
      dtype='object')

In [30]:
df[df['ACTION_TYPE']!='NO ACTION'].groupby('ACTION_TYPE')['Customer_Id'].nunique()

ACTION_TYPE
TYPE 1    25780
TYPE 2       69
TYPE 3      859
Name: Customer_Id, dtype: int64

In [31]:
25780+69+859

26708

In [32]:
df.groupby('ACTION_TYPE')['Customer_Id'].count()

ACTION_TYPE
NO ACTION      162
TYPE 1       25781
TYPE 2          69
TYPE 3         859
Name: Customer_Id, dtype: int64

In [33]:
25781+69+859+162

26871

In [34]:
len(df['Customer_Id'].unique())

26870